# 03 — Analyse des loyers (Île-de-France)

Notebook interactif pour explorer le fichier de loyers `pred-app12-mef-dhup_2024.csv`.
**Objectifs** : charger, nettoyer, filtrer IDF, visualiser distribution et proposer un widget pour lister
communes dont le loyer prédit est sous la moyenne (ou selon un seuil choisi).


In [ ]:

import os, warnings
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets, HBox, VBox, fixed
warnings.filterwarnings('ignore')

# Chemins — modifiables
default_paths = [
    os.path.join("data","raw","pred-app12-mef-dhup_2024.csv"),
    os.path.join("..","data","raw","pred-app12-mef-dhup_2024.csv"),
    r"C:/Users/Victor/DataScience/Projet-Data-science-Investissement-immobilier/pred-app12-mef-dhup_2024.csv"
]

csv_path = None
for p in default_paths:
    if os.path.exists(p):
        csv_path = p; break

print('Chemin utilisé pour le fichier loyers :', csv_path or "Aucun fichier trouvé automatiquement — modifiez csv_path manuellement")


## 1) Chargement et nettoyage initial
- Lecture robuste (latin-1 ou utf-8)
- Conversion du code INSEE en str
- Extraction numérique du champ `loypredm2` et création de `loy_num`


In [ ]:

# Lecture flexible
if csv_path is None:
    raise FileNotFoundError("Aucun fichier de loyers trouvé automatiquement. Déposez pred-app12-mef-dhup_2024.csv dans data/raw/ ou modifiez csv_path.")

# Essayer plusieurs encodages/sep si nécessaire
try:
    df_raw = pd.read_csv(csv_path, sep=None, engine="python", dtype=str, encoding="utf-8")
except Exception as e:
    df_raw = pd.read_csv(csv_path, sep=None, engine="python", dtype=str, encoding="latin-1")

print("Taille initiale:", df_raw.shape)
display(df_raw.head(3))


In [ ]:

# Standardisation colonnes (vérifie noms)
print("Colonnes disponibles:", list(df_raw.columns))

# On s'assure que la colonne INSEE_C existe
possible_insee = [c for c in df_raw.columns if c.lower().startswith("insee") or c.lower().startswith("commune")]
print("Colonnes candidates INSEE/commune:", possible_insee)


In [ ]:

# Nettoyage / extraction numérique loypredm2 -> loy_num
col_loy = None
for c in df_raw.columns:
    if 'loy' in c.lower() and 'm2' in c.lower():
        col_loy = c; break
if col_loy is None:
    # heuristique: trouver colonnes contenant 'loypred' ou 'loy'
    for c in df_raw.columns:
        if 'loypred' in c.lower() or 'loy' in c.lower():
            col_loy = c; break

if col_loy is None:
    raise ValueError("Impossible de détecter une colonne de loyer. Vérifie le fichier source.")

print('Colonne loyer détectée :', col_loy)

df = df_raw.copy()
# Nettoyage du champ loyer : remplacer virgule, extraire float
df['loy_raw'] = df[col_loy].astype(str)
df['loy_num'] = df['loy_raw'].str.replace(',', '.', regex=False).str.extract(r'([0-9]+\.?[0-9]*)')[0].astype(float)
# colonne INSEE
insee_candidates = [c for c in df.columns if c.lower().startswith('insee')]
if len(insee_candidates)>0:
    df['INSEE_C'] = df[insee_candidates[0]].astype(str).str.strip()
else:
    # tenter 'CODGEO' ou 'COM' ou 'INSEE_C'
    for c in ['CODGEO','COM','code_commune','code_INSEE','INSEE_C']:
        if c in df.columns:
            df['INSEE_C'] = df[c].astype(str).str.strip(); break

print('Total lignes après extraction loy_num:', len(df))
df[['INSEE_C','loy_raw','loy_num']].head(5)


## 2) Filtrer Île-de-France
On considère comme préfixes IDF : 75,77,78,91,92,93,94,95 (code départemental en début du code INSEE / COM).


In [ ]:

# Filtrage IDF
idf_prefix = ('75','77','78','91','92','93','94','95')

if 'INSEE_C' not in df.columns:
    raise ValueError("Colonne INSEE_C manquante. Vérifie le fichier source ou adapte le notebook.")

df['INSEE_C'] = df['INSEE_C'].astype(str).str.zfill(5)
df_idf = df[df['INSEE_C'].str.startswith(idf_prefix, na=False)].copy()
print(f"Lignes totales: {len(df)} | Lignes IDF: {len(df_idf)}")

# moyenne IDF
mean_val = df_idf['loy_num'].mean(skipna=True)
print(f"Moyenne loypredm2 IDF: {mean_val:.2f} €/m²")


## 3) Visualisations (Matplotlib uniquement)
- Distribution des loyers (histogramme)
- Boxplot par département
- Carte non incluse dans ce notebook (sera faite dans notebook gares si besoin)


In [ ]:

# Distribution loyers
plt.figure()
plt.hist(df_idf['loy_num'].dropna(), bins=50)
plt.title('Distribution des loyers prédits (€/m²) — IDF')
plt.xlabel('€/m²'); plt.ylabel('Nombre de lignes')
plt.show()


In [ ]:

# Moyenne par département (par code INSEE -> département = 2 premiers digits)
df_idf['dept'] = df_idf['INSEE_C'].str[:2]
dept_stats = df_idf.groupby('dept')['loy_num'].agg(['count','mean','median']).reset_index().sort_values('mean', ascending=False)
plt.figure()
plt.bar(dept_stats['dept'], dept_stats['mean'])
plt.title('Loyer moyen (€/m²) par département — IDF')
plt.xlabel('Département'); plt.ylabel('Loyer moyen €/m²')
plt.show()

display(dept_stats.head(20))


## 4) Widgets interactifs
- Sélecteur de département
- Slider de seuil (<= moyenne par défaut)
- Bouton pour exporter la shortlist


In [ ]:

# Widgets
dept_options = ['(Tous)'] + dept_stats['dept'].tolist()
dept_dd = widgets.Dropdown(options=dept_options, description='Dépt:')
seuil = widgets.FloatSlider(value=mean_val, min=0, max=max(df_idf['loy_num'].dropna().max(), mean_val*2), step=0.5, description='Seuil €/m²:')
topn = widgets.IntSlider(value=20, min=5, max=200, step=5, description='Top N:')

def shortlist(dept, seuil_val, topn_val):
    d = df_idf.copy()
    if dept != '(Tous)':
        d = d[d['INSEE_C'].str.startswith(dept)]
    d = d[d['loy_num'] <= seuil_val].copy()
    # agrégation commune si colonne existante
    comm_col = None
    for c in ['commune', 'nom_commune', 'NOM_COM', 'LIBGEO', 'LIBELLE_COMMUNE']:
        if c in d.columns:
            comm_col = c; break
    if comm_col is None:
        # regroupe sur INSEE
        grp = d.groupby('INSEE_C').agg(nb=('loy_num','count'), loy_med=('loy_num','median')).reset_index().sort_values('loy_med')
    else:
        grp = d.groupby(comm_col).agg(nb=('loy_num','count'), loy_med=('loy_num','median')).reset_index().sort_values('loy_med')
    display(grp.head(topn_val))
    return grp.head(topn_val)

out = widgets.interactive_output(shortlist, {'dept':dept_dd, 'seuil_val':seuil, 'topn_val':topn})
display(HBox([dept_dd, seuil, topn]))
display(out)


## 5) Exporter les résultats filtrés
Tu peux sauvegarder la shortlist (CSV) sur disque.


In [ ]:

# Fonction d'export
def export_shortlist(df_short, fname='shortlist_loyers_idf.csv'):
    outp = os.path.join('data','clean', fname)
    os.makedirs(os.path.dirname(outp), exist_ok=True)
    df_short.to_csv(outp, index=False, encoding='utf-8-sig')
    print('Exporté →', outp)

# Exemple d'usage :
# grp = shortlist('(Tous)', mean_val, 20)
# export_shortlist(grp)
print('Cellule prête pour export.')
